In [1]:
import re
from collections import Counter
import random

In [2]:
"=====CONSTANTS====="
BOOK_FNAME = "AnnaKarenina.txt"
cyrillic_symbols = list("абвгдеёжзийклмнопрстуфхцчшщъыьэюя")
msg = "меня зовут максимус деций меридий командующий северными армиями генерал легиона феликс верный слуга истинного императора марка аврелия отец убитого сына муж убитой жены и я отомщу за них в этой жизни или следующей!"

**1. Реализуйте базовый частотный метод по Шерлоку Холмсу:**

подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);

возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;

расшифруйте их таким частотным методом.

In [3]:
def create_row_from_file():
  with open(BOOK_FNAME, "r") as fin:
    text = fin.readlines()
  one_row = " ".join(text)
  cleaned_row = re.sub(r"\W+", " ", one_row.lower())
  return cleaned_row
prepared_text = create_row_from_file()
letter_freqs = Counter(prepared_text)

In [4]:
def get_accuracy(msg, decoded_msg):
    match = 0
    for x, y in zip(msg, decoded_msg):
        if x == y:
            match += 1
    return match / len(msg)

def encode(msg, row):
    lett_list = list(row)
    random.shuffle(row)
    translation = str.maketrans("".join(lett_list), "".join(row))
    return msg.translate(translation)

def decode(encoded_msg, freqs, key):
    if key == 'word':
      enc_freqs = Counter(encoded_msg)
    elif key == 'bigram':
      enc_freqs = get_bigram_freqs(encoded_msg)
    sort_enc_freqs = [item for item, _ in sorted(enc_freqs.items(), key=lambda letter: (-letter[1], letter[0]))]
    sort_freqs = [item for item, _ in sorted(freqs.items(), key=lambda freq: (-freq[1], freq[0]))]
    translation = str.maketrans("".join(sort_enc_freqs), "".join(sort_freqs[:len(sort_enc_freqs)]))
    return encoded_msg.translate(translation)

def get_bigram_freqs(row):
    bigram_freqs = Counter()
    for idx in range(len(row) - 1):
        bigram_freqs[row[idx:idx + 2]] += 1
    return bigram_freqs

def print_stat(msg, enc, dec):
  print(f"Сообщение:\t{msg}")
  print(f"Закодированная строка:\t{enc}")
  print(f"Раскодированная строка:\t{dec}")
  print(f"Метрика: {get_accuracy(msg, dec)}")


In [5]:
enc_ru_msg = encode(msg, cyrillic_symbols)
dec_ru_msg = decode(msg, letter_freqs, 'word')
print_stat(msg, enc_ru_msg, dec_ru_msg)

Сообщение:	меня зовут максимус деций меридий командующий северными армиями генерал легиона феликс верный слуга истинного императора марка аврелия отец убитого сына муж убитой жены и я отомщу за них в этой жизни или следующей!
Закодированная строка:	юлцз чщиуг юёнжфюуж алсфо юлкфафо нщюёцаубпфо жлилкцйюф ёкюфзюф ялцлкёы ыляфщцё элыфнж илкцйо жыуяё фжгфццщящ фюьлкёгщкё юёкнё ёиклыфз щглс умфгщящ жйцё юуе умфгщо елцй ф з щгщюпу чё цфх и ъгщо ефчцф фыф жылаубпло!
Раскодированная строка:	иеты бнмлр иаявоилв пежок иесопок яниатплйчок веместьио асиоыио уетесад деуонта юедояв местьк вдлуа овроттнун оихесарнса иасяа амседоы нреж лзорнун вьта илг лзорнк геть о ы нрничл ба тоэ м щрнк гобто одо вдеплйчекш
Метрика: 0.2850467289719626


**2. Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:**

подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;

проведите тестирование аналогично п.1, но при помощи биграмм.



In [6]:
bigram_ru_freqs = get_bigram_freqs(prepared_text)
dec_birgam_msg = decode(enc_ru_msg, bigram_ru_freqs, 'bigram')
print_stat(msg, enc_ru_msg, dec_birgam_msg)

Сообщение:	меня зовут максимус деций меридий командующий северными армиями генерал легиона феликс верный слуга истинного императора марка аврелия отец убитого сына муж убитой жены и я отомщу за них в этой жизни или следующей!
Закодированная строка:	юлцз чщиуг юёнжфюуж алсфо юлкфафо нщюёцаубпфо жлилкцйюф ёкюфзюф ялцлкёы ыляфщцё элыфнж илкцйо жыуяё фжгфццщящ фюьлкёгщкё юёкнё ёиклыфз щглс умфгщящ жйцё юуе умфгщо елцй ф з щгщюпу чё цфх и ъгщо ефчцф фыф жылаубпло!
Раскодированная строка:	мдянжммолежмвмаумлажыдлутжмдвуыутжмммвяыл еутжадодвя мужввмунмужтдядввыжыдтумявжедыумажодвя тжаылтвжуаеуяямтмжумадввемввжмввмвжвовдыунжмедлжлоуемтмжа явжмлсжлоуемтжсдя жужнжмеммелжмвжяусжожыемтжсумяужуыужаыдыл едти
Метрика: 0.056074766355140186
